<font size = 6> About this File:<br>
<font size = 3.5> A simple RAG (Retrieval-Augmented Generation). <br>
Documents and tools:
* retrieval knowledge base: the SNA 2008 manual (System of National Accounts)
* LLM: ChatGPT 3.5 Instruct
* Embedding model: "text-embedding-ada-002" (ChatGPT)
* Vector database: pinecone (serverless)

<br><br>Example query: "_In what situations do national accounts consolidate accounts?_", <br>


# Keys

In [ ]:
# github version

import os

PINECONE_API_KEY = os.environ["api_key_openai"]
OPENAI_API_KEY = os.environ["api_key_pinecone"]

# Import libraries

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pdfplumber
import pypdf
import PyPDF2
from IPython.display import display

# Data Preparation

## Parse SNA Manual - PDF Document - with PDF reader libraries

### Try few different parsers

In [2]:
PDF_PATH = r"C:\Users\xzhan\Documents\Vertical Definition and Views\SNA 2008 Manual.pdf"

#### pdfplumber<br>
This one has problem with dual columns

In [4]:
# By page; this one does not handle dual column; it's a no go

import pdfplumber

# from tqdm import tqdm


pdf_path = r"C:\Users\xzhan\Documents\Vertical Definition and Views\SNA 2008 Manual.pdf"
text_path = r"C:\github\chatgpt\sna_manual_parsed_text.txt"


with pdfplumber.open(pdf_path) as pdf:

    full_text = ""

    for page in pdf.pages:

        # Extract text from each page and append it to the full_text string

        text = page.extract_text()

        if text:  # Checking if text extraction returned any content

            full_text += text + " "


with open(text_path, "w", encoding="utf-8") as text_file:
    text_file.write(full_text)

In [6]:
# By line and removing \n; this one does not handle dual column; it's a no go

import pdfplumber

# from tqdm import tqdm


with pdfplumber.open(pdf_path) as pdf:

    full_text = ""

    for page in tqdm(pdf.pages, desc="Processing Pages"):
        text = page.extract_text()
        if text:
            # Replace singular newline characters with a space, assuming double newlines are paragraph breaks
            text = text.replace(
                "\n", " "
            )  # This replaces all newlines; might need more logic for double newlines
            full_text += text + " "

with open(text_path, "w", encoding="utf-8") as text_file:
    text_file.write(full_text)

Processing Pages: 100%|██████████| 722/722 [01:36<00:00,  7.50it/s]


#### UnstructuredPDFLoader
* Too many dependencies; not worth the effort given the current goals (more just texts)

In [ ]:
# too many dependencies, not worth doing it now.

from langchain_community.document_loaders import UnstructuredPDFLoader

pdf_path = PDF_PATH
loader = UnstructuredPDFLoader(pdf_path)
data = loader.load()
data

#### pypdf (PyPFD 1 version)

In [16]:
from langchain_community.document_loaders import PyPDFLoader

text_path = r"C:\github\chatgpt\sna_manual_parsed_text.txt"

loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

### pypdf2
* Ended up using this one b/c it can handle dual column layout in the pdf file
* Relatively easy to use and stable
* Cons: some words were separated by " " - pdf layout caused issues; could not extract table data well

In [24]:
import PyPDF2

pdf_path = PDF_PATH  # Update this to the path of your PDF file
text_path = r"C:\github\chatgpt\sna_manual_parsed_text_pypdf2.txt"

# Open the PDF file
with open(pdf_path, "rb") as file:
    reader = PyPDF2.PdfReader(file)
    text = []

    # Iterate over each page and extract text
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text.append(page.extract_text())  # extract_text() is the updated method name

    # Combine the text of all pages into a single string
    full_text = "\n".join(text)

# Now you can print the text or save it to a file
with open(text_path, "w", encoding="utf-8") as text_file:
    text_file.write(full_text)

In [34]:
# Cleaning "......"

cleaned_lines = []
with open(text_path, "r", encoding="utf-8") as file:
    lines = file.readlines()
    for line in lines:
        # Your line processing logic
        if not any(char.isalnum() for char in line) or line.count(".") > 5:
            continue
        cleaned_lines.append(line)

with open(text_path, "w", encoding="utf-8") as file:
    file.writelines(cleaned_lines)

In [35]:
# Remove the line breaks to form a single blob of texts
cleaned_text = ""
with open(text_path, "r", encoding="utf-8") as file:
    lines = file.readlines()
    for line in lines:
        # Remove unwanted lines based on your previous criteria
        if not any(char.isalnum() for char in line) or line.count(".") > 5:
            continue
        # Strip trailing newline characters and whitespace, then concatenate
        cleaned_text += line.strip() + " "  # Add a space for word separation

# Now cleaned_text contains all the cleaned lines stitched together
# Optionally, you can save this back to a file
with open(text_path, "w", encoding="utf-8") as file:
    file.write(cleaned_text)

## Parse text into sentences via Spacy

### Spacy split into sententences & save to a file - Optional

In [ ]:
# This is optional, if you want to do this more in multiple discrete steps
import spacy
from tqdm import tqdm


# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define the path for your input and output files
input_file_path = r"C:\github\chatgpt\sna_manual_parsed_text_pypdf2.txt"
output_file_path = r"C:\github\chatgpt\sna_manual_parsed_sents.txt"


# Read the large text file in chunks (you can adjust the chunk size)
def read_in_chunks(file_path, chunk_size=1000):
    with open(file_path, "r", encoding="utf-8") as file:
        while True:
            chunk = file.read(chunk_size)

            if not chunk:
                break
            yield chunk
        # the while loop creates an infinite loop.
        # This loop will continuously execute its block of code until it encounters a break statement or another form of interruption.
        # This condition becomes true when file.read(chunk_size) reaches the end of the file and returns an empty string.


# Initialize a progress bar
file_size = sum(1 for _ in open(input_file_path, "r", encoding="utf-8"))
# The sum(1 for ...) iterates over each line in the input file and sums up a count of 1 for each line,
# counting the total number of lines in the file.
# "each line" refers to a single line of text in the input file, as separated by newline characters (\n).

pbar = tqdm(total=file_size, desc="Processing")
# The value is used to set the maximum value (total) for the progress bar.


# Process the text in batches and write sentences to the output file
with open(output_file_path, "w", encoding="utf-8") as output_file:
    for text_chunk in read_in_chunks(input_file_path):
        # Processing the chunk with spaCy
        for doc in nlp.pipe(
            [text_chunk], batch_size=100
        ):  # Adjust batch_size as needed
            for sent in doc.sents:

                output_file.write(sent.text + "\n")

        pbar.update(len(text_chunk))  # Update progress after each chunk is processed


pbar.close()  # Close the progress bar

In [ ]:
# This is optional; quick count of how many lines of sentences


def count_lines(filename):
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.readlines()
    return len(lines)


print(count_lines(output_file_path))

### spaCy split into sents, add IDs, & save to a json file format (pinecone ready format)<br>
* About spaCy: developed by MIT, spaCy is a library for Natural Language Processing in Python. I am using its quick text processing feature to parse the text into sentences. This is more of a personal choice. spaCy can condenses few lines of codes into one or two lines of code with multiple generators. However, it is more "high maintenance" when it comes to installation and maintenance... a lot pip re-installs over the years. Therefore, the NLTK library, by Columbia University, is probably safer to use, as it is more popular and "stable". 
*  I included an extra step to add an "sent-id-xxx" field. Although adding ids is not required, but it's a good practice.

In [1]:
import spacy
from tqdm import tqdm
import json


# Load the spaCy language model
nlp = spacy.load("en_core_web_sm")

# Define the path for the input and output files
input_file_path = r"C:\github\chatgpt\sna_manual_parsed_text_pypdf2.txt"
output_json_path = r"C:\github\chatgpt\sna_manual_parsed_pinecone_data.json"


# Read the large text file in chunks (you can adjust the chunk size in the function)
def read_in_chunks(file_path, chunk_size=1000):
    with open(file_path, "r", encoding="utf-8") as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            yield chunk
        # the while loop creates an infinite loop.
        # This loop will continuously execute its block of code until it encounters a break statement or another form of interruption.
        # This condition becomes true when file.read(chunk_size) reaches the end of the file and returns an empty string.


# Initialize a progress bar
total_progress = sum(1 for _ in open(input_file_path, "r", encoding="utf-8"))
# The sum(1 for ...) iterates over each line in the input file and sums up a count of 1 for each line,
# counting the total number of lines in the file.
# "each line" refers to a single line of text in the input file, as separated by newline characters (\n).
pbar = tqdm(total=total_progress, desc="Processing")
# The value is used to set the maximum value (total) for the progress bar.

# Process the text in batches load into a variable
all_sents = []
for text_chunk in read_in_chunks(input_file_path):
    # Processing the chunk with spaCy
    for doc in nlp.pipe([text_chunk], batch_size=100):  # Adjust batch_size as needed
        for sent in doc.sents:
            all_sents.append(sent.text.strip())
            pbar.update(
                len(text_chunk)
            )  # Update progress after each chunk is processed

# Adjust the progress bar's total to account for the remaining steps (saving to JSON)
additional_steps = len(
    all_sents
)  # Assuming each sentence will count as a step in the progress
pbar.total += additional_steps

# Create a list of dictionaries for Pinecone upsert
pinecone_data = [
    {"id": f"sent-id-{i + 1}", "text": sentence} for i, sentence in enumerate(all_sents)
]
pbar.update(additional_steps)  # Update progress for appending sentences

# save data to a JSON file
with open(output_json_path, "w", encoding="utf-8") as json_file:
    json.dump(pinecone_data, json_file)
    pbar.update(len(all_sents))

pbar.close()

Processing: 22982323it [01:01, 375110.93it/s]    


## Read Data and Chunk<br>
* Pinecone vector database requires at least id (unique identifier) and value (vector values, i.e. [0.5, 1, 1.2...]) to upload. Therefore, we must generate unique ids for the "chunked" texts. 

In [3]:
from tqdm.auto import tqdm
import json

json_path = r"C:\github\chatgpt\sna_manual_parsed_pinecone_data.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)


window = 10  # No. of sentences to combine
stride = 2  # No. of sentences to 'stride' over
# (you want some "buffer" before and after each "text snippet."
# Snippet 2 overlaps with snippet 1, and 3 some overlap with 2, so on...
# The redundancy makes earch snippet a bit richer.)

chunked_data = []  # Store the resulting chunks here

for i in tqdm(range(0, len(data), stride)):
    i_end = min(len(data), i + window)
    chunk_text = " ".join(
        item["text"] for item in data[i:i_end]
    )  # Aggregate text within the window

    # Create and append the chunked data
    # Assign a new ID or use the ID of the first sentence in the chunk
    chunk_id = f"chunk-id-{i // stride}"
    chunked_data.append({"id": chunk_id, "text": chunk_text})

  0%|          | 0/11469 [00:00<?, ?it/s]

In [3]:
len(chunked_data)

11469

## Embed and upsert (load) into vector database

In [4]:
# Instantiate pinecone (vector dbs/vecotor store)

import pinecone
from pinecone import Pinecone, ServerlessSpec, PodSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
spec = ServerlessSpec(cloud="aws", region="us-west-2")
# pinecone.whoami()

In [5]:
index_name = "rag-qa-sna-manual"  # name of the vector dbs

In [7]:
# Instantiate pinecone index (vector dbs) (create a new one if we have not yet built one)


# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-west-2"),
    )
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

### Upsert data into pinecone
* For our purpose, we also need text associated with the values in pinecone's metadata (we use the vector to search, but it's the snippets of document texts that we need.) Therefore, we need to include text on top of ids and values.

In [33]:
# This is the version modified by chatgpt
# This part takes few minutes, depending on your hardware

import openai
from tqdm.auto import tqdm
import time

embed_model = "text-embedding-ada-002"
openai.api_key = OPENAI_API_KEY
data = chunked_data
batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i + batch_size)
    meta_batch = data[i:i_end]

    ids_batch = [x["id"] for x in meta_batch]
    texts = [x["text"] for x in meta_batch]

    embeds = []
    attempt = 0
    max_attempts = 5
    while not embeds and attempt < max_attempts:
        try:
            res = openai.embeddings.create(input=texts, model=embed_model)
            embeds = [embedding.embedding for embedding in res.data]
            # [embedding["embedding"] for embedding in res.data]
        except Exception as e:  # Catching any exception
            print(f"An error occurred: {e}")
            # Implementing a simple backoff strategy, assuming error might be rate-limiting or similar
            sleep_time = 60 * (2**attempt)
            time.sleep(sleep_time)
            attempt += 1

    if not embeds:
        print(f"Failed to create embeddings after {max_attempts} attempts.")
        continue

    to_upsert = [
        (id, embed, {"text": text}) for id, embed, text in zip(ids_batch, embeds, texts)
    ]  # this upserts id, values (vectors), and text

    index.upsert(vectors=to_upsert)

  0%|          | 0/115 [00:00<?, ?it/s]

# Query augmented by retrieval

In [45]:
query = "In what situations do national accounts consolidate accounts?"

## Query without retrieval augmentation

In [46]:
import openai

openai.api_key = OPENAI_API_KEY
openai.models.list()


def complete(prompt):
    res = openai.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=0,
        max_tokens=400,
        top_p=1,
        frequency_penalty=0,
        stop=None,
    )
    return res.choices[0].text.strip()


answer_no_rag = complete(query)
print(answer_no_rag)

National accounts typically consolidate accounts in the following situations:

1. Government accounts: National accounts consolidate the accounts of different levels of government (federal, state, and local) to provide a comprehensive view of the country's public finances.

2. International trade: National accounts consolidate the accounts of imports and exports to measure a country's balance of trade and current account balance.

3. Financial sector: National accounts consolidate the accounts of financial institutions, such as banks and insurance companies, to measure the country's financial sector's contribution to the economy.

4. Public and private sectors: National accounts consolidate the accounts of both the public and private sectors to provide a complete picture of the country's economic activity.

5. National income: National accounts consolidate the accounts of different sectors of the economy (households, businesses, government) to measure the country's national income and 

## Create a query vector, search the retrieval vector dbs (pinecone)

### Example of how to search the retrieval vector dbs

In [35]:
res = openai.embeddings.create(input=[query], model=embed_model)

# retrieve the embedding
xq = res.data[0].embedding

# get relevant contexts (including the questions)
index = pc.Index(index_name)
res = index.query(vector=xq, top_k=2, include_metadata=True)

In [36]:
res

{'matches': [{'id': 'chunk-id-590',
              'metadata': {'text': 'As a matter of principle, flows between '
                                   'constituent units within subsectors or  '
                                   'sectors are not consolidated. However, '
                                   'consolidated account s may be compiled for '
                                   'complementary presentations and analyses. '
                                   'Even then, transactionsappearing in '
                                   'different accounts are never consolidated '
                                   'so that the balancing items are not '
                                   'affected by consolidation. Consolidation '
                                   'may be useful, for example, for the '
                                   'government sector as a whole, thus showing '
                                   'the net relations between g overnment and '
                            

### Search retrieval, augment, and query again

In [47]:
# edited by ChatGPT
limit = 5000


def retrieve(query):
    res = openai.embeddings.create(input=[query], model=embed_model)

    # retrieve from Pinecone
    xq = res.data[0].embedding

    # get relevant contexts
    res = index.query(vector=xq, top_k=3, include_metadata=True)
    contexts = [x["metadata"]["text"] for x in res["matches"]]

    # build your pompt w/t the retreived context included
    prompt_start = "Answer the question based on the context below.\n\n" + "Context:\n"
    prompt_end = f"\n\nQuestion: {query}\nAnswer:"

    # # Initialize prompt to avoid UnboundLocalError
    # prompt = prompt_start + prompt_end

    # append contexts until hitting limit
    prompt = prompt_start
    for context in contexts:
        if len(prompt + context + "\n\n---\n\n" + prompt_end) > limit:
            break
        prompt += context + "\n\n---\n\n"
    prompt += prompt_end

    return prompt

In [48]:
query_with_contexts = retrieve(query)
print(query_with_contexts)

Answer the question based on teh context below.

Context:
As a matter of principle, flows between constituent units within subsectors or  sectors are not consolidated. However, consolidated account s may be compiled for complementary presentations and analyses. Even then, transactionsappearing in different accounts are never consolidated so that the balancing items are not affected by consolidation. Consolidation may be useful, for example, for the government sector as a whole, thus showing the net relations between g overnment and the rest of the economy . This possibility is elaborated in chapter 22. 2.70 Accounts for the total econom y, when fully consolidated, give rise to the rest of the world account (external transactions account). Overview 23Netting 2.71 Consolidation must be di stinguished from netting. For current transactions, netting refers to offsetting uses against resources. The SNA does th is only in a few specific instances; for example, taxes on products may be shown 

In [49]:
# complete the context-infused query
answer_rag = complete(query_with_contexts)
print(answer_rag)

National accounts consolidate accounts in situations where it is useful for complementary presentations and analyses, such as for the government sector as a whole, to show the net relations between government and the rest of the economy.
